In [ ]:
import numpy as np

In [ ]:
# Se definen 10 comentarios de algún restaurante
reviews = [
          'Never coming back!',
          'horrible service',
          'rude waitress',
          'cold food',
          'horrible food!',
          'awesome',
          'awesome services!',
          'rocks',
          'nice work',
          'couldn\'t have done better'
]

# Se definen sus etiquetas. 1 para positivo, 0 para negativo
labels = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

#### Tokenización

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
"""Proceso de tokenizacion, preprocesamiento. Para tokenize"""
vocab_size = 50 #Tamaño del vocabulario, se puede usar con spacy. Cuantas palabras hay
oov_tok = '<OOV>' #Hace alución para out of vocabulary (es para las palabras que no se pueden usar). Fuera del vocabulario
padding_type = 'post' #Parecido a las redes neuronales convulucionales, es como crear el borde oscuro.

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok) #Taño del vocabulario y como nombrar a las palbras que quedarán fuera del vocabulario
tokenizer.fit_on_texts(reviews) #Pasamos la data limpia, ya normalizada
word_index = tokenizer.word_index #Extraemos atributos, donde se asocia a los tokens con un index

In [ ]:
word_index

{'<OOV>': 1,
 'horrible': 2,
 'food': 3,
 'awesome': 4,
 'never': 5,
 'coming': 6,
 'back': 7,
 'service': 8,
 'rude': 9,
 'waitress': 10,
 'cold': 11,
 'services': 12,
 'rocks': 13,
 'nice': 14,
 'work': 15,
 "couldn't": 16,
 'have': 17,
 'done': 18,
 'better': 19}

In [ ]:
tokenizer.word_counts #Conteo de palabras

OrderedDict([('never', 1),
             ('coming', 1),
             ('back', 1),
             ('horrible', 2),
             ('service', 1),
             ('rude', 1),
             ('waitress', 1),
             ('cold', 1),
             ('food', 2),
             ('awesome', 2),
             ('services', 1),
             ('rocks', 1),
             ('nice', 1),
             ('work', 1),
             ("couldn't", 1),
             ('have', 1),
             ('done', 1),
             ('better', 1)])

In [ ]:
vocab_size = len(word_index)
vocab_size #numero de palabras que conforman el corpues

19

In [ ]:
tokenized_reviews = tokenizer.texts_to_sequences(reviews)
tokenized_reviews #Indices asociados a las palabras

[[5, 6, 7],
 [2, 8],
 [9, 10],
 [11, 3],
 [2, 3],
 [4],
 [4, 12],
 [13],
 [14, 15],
 [16, 17, 18, 19]]

In [ ]:
max_length = max([len(x) for x in tokenized_reviews])
max_length #Maxima lonngitud de la secuencia mas larga

4

In [ ]:
padded_reviews = pad_sequences(tokenized_reviews, maxlen=max_length, padding=padding_type)
print(padded_reviews) #Rellenamos con ceros la indexacion con el tamaño de la secuencia más larga

[[ 5  6  7  0]
 [ 2  8  0  0]
 [ 9 10  0  0]
 [11  3  0  0]
 [ 2  3  0  0]
 [ 4  0  0  0]
 [ 4 12  0  0]
 [13  0  0  0]
 [14 15  0  0]
 [16 17 18 19]]


#### Word Embeddings

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense

In [ ]:
"""Creamos nuestros word Embeddings."""
embedding_layer = Embedding(input_dim=vocab_size+1, output_dim=8, input_length=max_length) #Capa nueva
#Dimension de entrada tamaño del vocabulario más un valor adicional para palabras fuera del vocabulario
#output dim determina cuanto va a tardar nuestro algoritmo a ser entrenado, son el numero de variables del vector continuo, buen tamaño 100
#longitud de entrada es la longitud maxima

In [ ]:
model = Sequential() #Instanciamos
model.add(embedding_layer) #Capa de incrustaciones
model.add(Flatten()) # Desenrrollamos, no se recomienda, se puede usar otras arquitecturas
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) #Clasificación binaria

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              160       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.6910 - accuracy: 0.6000
Epoch 2/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6892 - accuracy: 0.6000
Epoch 3/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6874 - accuracy: 0.7000
Epoch 4/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6857 - accuracy: 0.7000
Epoch 5/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6839 - accuracy: 0.7000
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6821 - accuracy: 0.9000
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6803 - accuracy: 0.9000
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6785 - accuracy: 0.9000
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6768 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6750 - accuracy: 1.0000
Epoch 11/100
1/1 [=======

In [ ]:
import pandas as pd
from scipy import spatial

In [ ]:
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y) #Similitud cosenoidal

In [ ]:
print(embedding_layer.get_weights()[0].shape) #matriz de incrustaciones, observacion de la matriz

(20, 8)


In [ ]:
pd.DataFrame((embedding_layer.get_weights()[0]), index=['<pad>']+list(word_index.keys()), columns=['dim_'+str(x) for x in range(1,9)])
#Vector continuo asociado a las palabras, similar a PCA

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8
<pad>,-0.030751,-0.130159,0.121297,0.054825,-0.128418,0.063643,0.150049,-0.123110
<OOV>,0.038733,-0.028142,-0.022666,-0.022424,0.002987,0.005259,-0.026361,0.038195
horrible,-0.102366,-0.111475,0.093545,-0.160318,-0.143454,-0.092147,-0.085164,-0.068161
food,0.076558,0.142790,-0.148010,-0.124077,0.079903,0.128303,-0.123759,0.151828
awesome,0.101571,0.066223,-0.068894,0.149145,0.107159,0.108261,0.140760,0.129726
never,-0.076121,-0.131378,0.069915,-0.089701,-0.156249,-0.077513,-0.154104,-0.060421
coming,0.095250,0.132554,-0.136284,-0.073360,0.105109,0.136303,-0.062908,0.147670
back,-0.063298,0.156527,0.083629,-0.130288,0.111158,0.057961,0.118312,0.159569
service,0.156683,0.153118,-0.065421,-0.136383,0.094168,0.114992,-0.062776,0.121672
rude,-0.126471,-0.143259,0.129160,-0.078165,-0.133796,-0.104050,-0.074201,-0.116594


In [ ]:
"""Calculo de similitudes. me quede min 42"""

computed_similarities = pd.DataFrame()

for word1 in word_index.keys():
    for word2 in word_index.keys():
        idx1 = word_index[word1]
        idx2 = word_index[word2]
        embedding1 = embedding_layer.get_weights()[0][idx1]
        embedding2 = embedding_layer.get_weights()[0][idx2]
        similarity = cosine_similarity2(embedding1, embedding2)
        computed_similarities.loc[word1, word2] = similarity

In [ ]:
def non_zero_green(val):
    return 'background-color: Aquamarine' if (val > 0.9) & (val < 1) else ''

In [ ]:
computed_similarities.style.applymap(non_zero_green)

,,horrible,food,awesome,never,coming,back,service,rude,waitress,cold,services,rocks,nice,work,couldn't,have,done,better
,1.000000,-0.026866,0.574915,0.102471,0.090509,0.539410,-0.091150,0.544239,-0.226016,0.521528,-0.033361,-0.678108,0.124422,0.154133,-0.644669,0.165269,-0.611046,0.267406,-0.307213
horrible,-0.026866,1.000000,-0.368384,-0.943888,0.938738,-0.566785,-0.238115,-0.423789,0.940954,-0.385170,0.943249,0.418621,-0.978536,-0.927748,0.284343,-0.980759,0.381699,0.315982,0.361057
food,0.574915,-0.368384,1.000000,0.298980,-0.335874,0.970355,0.407232,0.923142,-0.623756,0.932963,-0.379245,-0.967346,0.422416,0.394771,-0.920897,0.393873,-0.938926,-0.220974,-0.287265
awesome,0.102471,-0.943888,0.298980,1.000000,-0.914900,0.513033,0.327913,0.372839,-0.901824,0.271144,-0.932040,-0.338051,0.954733,0.982025,-0.192428,0.957747,-0.282027,-0.381295,-0.466580
never,0.090509,0.938738,-0.335874,-0.914900,1.000000,-0.539842,-0.477117,-0.446305,0.917084,-0.391299,0.931532,0.345540,-0.896779,-0.899017,0.262984,-0.934167,0.341331,0.526293,0.163737
coming,0.539410,-0.566785,0.970355,0.513033,-0.539842,1.000000,0.454840,0.933992,-0.785454,0.921016,-0.572547,-0.949810,0.612932,0.589930,-0.888434,0.596495,-0.928947,-0.302195,-0.336632
back,-0.091150,-0.238115,0.407232,0.327913,-0.477117,0.454840,1.000000,0.505806,-0.388476,0.453042,-0.250386,-0.283779,0.189174,0.323636,-0.318086,0.277634,-0.336575,-0.891516,0.073485
service,0.544239,-0.423789,0.923142,0.372839,-0.446305,0.933992,0.505806,1.000000,-0.673064,0.962374,-0.408702,-0.915158,0.459719,0.444464,-0.964040,0.478060,-0.940688,-0.240134,-0.263682
rude,-0.226016,0.940954,-0.623756,-0.901824,0.917084,-0.785454,-0.388476,-0.673064,1.000000,-0.616953,0.929455,0.658159,-0.947989,-0.931025,0.540021,-0.948227,0.599535,0.352135,0.380753
waitress,0.521528,-0.385170,0.932963,0.271144,-0.391299,0.921016,0.453042,0.962374,-0.616953,1.000000,-0.331214,-0.929005,0.384947,0.327813,-0.969367,0.431398,-0.972917,-0.256005,-0.091362


#### Predicción

In [ ]:
reviews

['Never coming back!',
 'horrible service',
 'rude waitress',
 'cold food',
 'horrible food!',
 'awesome',
 'awesome services!',
 'rocks',
 'nice work',
 "couldn't have done better"]

In [ ]:
# Se definen comentarios adicionales sobre los que se requiere predecir
new_reviews  = ["Nice!",
               "rude service",
               "terrible food"]

# Se convierten a una secuencias
sequences = tokenizer.texts_to_sequences(new_reviews)

# Se rellenan las secuencias
new_padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type)

# Se predice el sentimiento
print(model.predict(new_padded), '\n', '----'*3)
print(np.where(model.predict(new_padded)>0.5, 1, 0))

[[0.61956495]
 [0.36223292]
 [0.43261477]] 
 ------------
[[1]
 [0]
 [0]]


#### RNN

In [ ]:
from tensorflow.keras.layers import SimpleRNN

In [ ]:
model_rnn = Sequential()
model_rnn.add(embedding_layer)
model_rnn.add(SimpleRNN(units=64))
model_rnn.add(Dense(1, activation='sigmoid'))

model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              160       
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                4672      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,897
Trainable params: 4,897
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_rnn.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 989ms/step - loss: 0.7445 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 18ms/step - loss: 0.7023 - accuracy: 0.3000
Epoch 3/100
1/1 [==============================] - 0s 16ms/step - loss: 0.6621 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6238 - accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 0.5869 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 0.5515 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 16ms/step - loss: 0.5171 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 14ms/step - loss: 0.4837 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 15ms/step - loss: 0.4510 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 12ms/step - loss: 0.4191 - accuracy: 1.0000
Epoch 11/100
1

In [ ]:
print(model_rnn.predict(new_padded), '\n', '----'*3)
print(np.where(model_rnn.predict(new_padded)>0.5, 1, 0))

[[0.9985367 ]
 [0.00121384]
 [0.04299283]] 
 ------------
[[1]
 [0]
 [0]]


#### GRU

In [ ]:
from tensorflow.keras.layers import GRU

In [ ]:
model_gru = Sequential()
model_gru.add(embedding_layer)
model_gru.add(GRU(units=64))
model_gru.add(Dense(1, activation='sigmoid'))

model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_gru.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              160       
                                                                 
 gru (GRU)                   (None, 64)                14208     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 14,433
Trainable params: 14,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_gru.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.6915 - accuracy: 0.4000
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6894 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6872 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 18ms/step - loss: 0.6849 - accuracy: 0.9000
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6827 - accuracy: 0.9000
Epoch 6/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6803 - accuracy: 0.9000
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6779 - accuracy: 0.9000
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6754 - accuracy: 0.9000
Epoch 9/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6728 - accuracy: 0.9000
Epoch 10/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6701 - accuracy: 0.9000
Epoch 11/100
1/1 [====

In [ ]:
print(model_gru.predict(new_padded), '\n', '----'*3)
print(np.where(model_gru.predict(new_padded)>0.5, 1, 0))

[[9.9938285e-01]
 [7.6375692e-04]
 [5.1964032e-03]] 
 ------------
[[1]
 [0]
 [0]]


#### LSTM

In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
model_lstm = Sequential()
model_lstm.add(embedding_layer)
model_lstm.add(LSTM(units=64))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_lstm.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              160       
                                                                 
 lstm (LSTM)                 (None, 64)                18688     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 18,913
Trainable params: 18,913
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_lstm.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.6955 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6916 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6878 - accuracy: 0.6000
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6839 - accuracy: 0.6000
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6800 - accuracy: 0.6000
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6761 - accuracy: 0.8000
Epoch 7/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6721 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6679 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6637 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6594 - accuracy: 1.0000
Epoch 11/100
1/1 [=====

In [ ]:
print(model_lstm.predict(new_padded), '\n', '----'*3)
print(np.where(model_lstm.predict(new_padded)>0.5, 1, 0))

[[0.99685705]
 [0.00231003]
 [0.0254944 ]] 
 ------------
[[1]
 [0]
 [0]]


#### Bidireccional

In [ ]:
from tensorflow.keras.layers import Bidirectional

In [ ]:
model_bd = Sequential()
model_bd.add(embedding_layer)
model_bd.add(Bidirectional(layer=LSTM(units=64)))
model_bd.add(Dense(1, activation='sigmoid'))

model_bd.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_bd.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              160       
                                                                 
 bidirectional (Bidirectiona  (None, 128)              37376     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 37,665
Trainable params: 37,665
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_bd.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.7159 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 13ms/step - loss: 0.7075 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 13ms/step - loss: 0.6992 - accuracy: 0.4000
Epoch 4/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6910 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6828 - accuracy: 0.6000
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 0.6747 - accuracy: 0.6000
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6665 - accuracy: 0.6000
Epoch 8/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6583 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 0.6500 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6416 - accuracy: 1.0000
Epoch 11/100
1

In [ ]:
print(model_bd.predict(new_padded), '\n', '----'*3)
print(np.where(model_bd.predict(new_padded)>0.5, 1, 0))

[[0.99805945]
 [0.00213523]
 [0.0276983 ]] 
 ------------
[[1]
 [0]
 [0]]


#### Convolución 1D

In [ ]:
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D

[Global Average Pooling](https://paperswithcode.com/method/global-average-pooling)

In [ ]:
model_conv = Sequential()
model_conv.add(embedding_layer)
model_conv.add(Conv1D(filters=8, kernel_size=3, activation='relu'))
model_conv.add(GlobalAveragePooling1D())
model_conv.add(Dense(1, activation='sigmoid'))

model_conv.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_conv.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              160       
                                                                 
 conv1d (Conv1D)             (None, 2, 8)              200       
                                                                 
 global_average_pooling1d (G  (None, 8)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 369
Trainable params: 369
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_conv.fit(padded_reviews, labels, epochs=100)

Epoch 1/100
1/1 [==============================] - 8s 8s/step - loss: 0.6880 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6816 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 0.6751 - accuracy: 0.6000
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6686 - accuracy: 0.8000
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6621 - accuracy: 0.9000
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6556 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 14ms/step - loss: 0.6491 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 0.6425 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 0.6360 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 8ms/step - loss: 0.6296 - accuracy: 1.0000
Epoch 11/100
1/1 [======

In [ ]:
print(model_conv.predict(new_padded), '\n', '----'*3)
print(np.where(model_conv.predict(new_padded)>0.5, 1, 0))

[[0.7862312 ]
 [0.1831048 ]
 [0.26949495]] 
 ------------
[[1]
 [0]
 [0]]
